<a href="https://colab.research.google.com/github/steinhaug/stable-diffusion/blob/main/BLIP-2/AutoCaptioner-postWorker.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [142]:
#@markdown load notebook functions
def fill_array_length(arr, length=10):
    length = length + 1
    if len(arr) < length:
        arr += [''] * (length - len(arr))
    return arr

def contains_any_substring(input_string, substrings, force_lowercase=True):
    for substring in substrings:
        if substring in input_string.lower():
            return True
    return False

def extract_first_string_in_quotes(input_string):
    start_quote = input_string.find('"')
    end_quote = input_string.find('"', start_quote + 1)

    if start_quote != -1 and end_quote != -1:
        contents_in_quotes = input_string[start_quote + 1:end_quote]
        return contents_in_quotes
    else:
        return None

def string__return_left_from_match(input_string, array_of_strings, replacement=''):
    for match in array_of_strings:
        if match in input_string:
            left_part = input_string.split(match)[0]
            return left_part + replacement
    return input_string

def remove_prefix(input_string, prefixes):
    for prefix in prefixes:
        if input_string.startswith(prefix):
            input_string = input_string[len(prefix):]

    return input_string

def remove_matches(main_string, substrings_to_remove):
    for substring in substrings_to_remove:
        main_string = main_string.replace(substring, '')
    return main_string

# Strip ordered list, number - prefix
def strip_ol(input_string):
    index_of_space = input_string.find('-')
    if index_of_space != -1:
        result_string = input_string[index_of_space + 1:]
        return result_string.strip()
    else:
        return input_string

def true_or_false(string, matches=["yes"]):
    if contains_any_substring(string, matches):
        return True
    else:
        return False


In [143]:
text = '''1 - If this was a photo for an adult movie, the title would be "Sultry Seduction."
2 - The subject is a beautiful, blonde woman with a captivating gaze.
3 - The woman is wearing a black bra and panties, which are visible in the image.
4 - The woman's pussy is shaved, giving her a clean and polished appearance.
5 - The woman has large breasts, which are prominently displayed in the photo.
6 - The woman has a normal body build, with a healthy and attractive physique.
7 - Yes
8 - Yes
9 - No
10 - Lying on her back
11 - The woman's hands are resting on her thighs, adding to the sensual and alluring atmosphere of the image.'''

In [144]:
text = text.strip()
lines = fill_array_length( text.split('\n') )
lineout = fill_array_length([])

In [145]:
lineout[0] = extract_first_string_in_quotes(lines[0])
print(lineout[0])

Sultry Seduction.


In [134]:
lineout[1] = remove_prefix(lines[1][4:], ["The subject is "]).capitalize()
print( lineout[1] )

A beautiful, blonde woman with a captivating gaze.


In [135]:
lineout[2] = remove_prefix(lines[2][4:], ["The subject is "])
if contains_any_substring(result, ["completely naked"]):
    lineout[2] = 'NSWF, totally naked.'.capitalize()
else:
    lineout[2] = remove_matches(result, [" as she is",", which are visible in the image"]).capitalize()

print( lineout[2] )

The woman is wearing a black bra and panties.


In [137]:
if contains_any_substring(strip_ol(lines[3]), ["not possible"]):
    result = '-'
elif contains_any_substring(strip_ol(lines[3]), ["shaved", "shaven"]):
    result = 'Shaved pussy.'
elif contains_any_substring(strip_ol(lines[3]), ["trimmed"]):
    result = 'Trimmed pussy.'
elif contains_any_substring(strip_ol(lines[3]), ["hairy"]):
    result = 'Hairy pussy.'
else:
    result = '-'

lineout[3] = result
print( lineout[3] )

Shaved pussy.


In [138]:
if contains_any_substring(strip_ol(lines[4]), ["not possible"]):
    result = '-'
elif contains_any_substring(strip_ol(lines[4]), ["tiny"]):
    result = 'Tiny tits, tiny breasts.'
elif contains_any_substring(strip_ol(lines[4]), ["small"]):
    result = 'Small tits, small breasts.'
elif contains_any_substring(strip_ol(lines[4]), ["normal"]):
    result = 'Normal tits, normal breasts.'
elif contains_any_substring(strip_ol(lines[4]), ["large"]):
    result = 'Large tits, large breasts.'
elif contains_any_substring(strip_ol(lines[4]), ["huge","big"]):
    result = 'Big tits, big breasts.'
else:
    result = '-'

lineout[4] = result
print( lineout[4] )

Large tits, large breasts.


In [139]:
if contains_any_substring(strip_ol(lines[5]), ["slender and slim"]):
    result = 'Slim figure.'
elif contains_any_substring(strip_ol(lines[5]), ["appears to be normal","normal body"]):
    result = 'Normal figure.'
elif contains_any_substring(strip_ol(lines[5]), ["thin"]):
    result = 'Thin figure.'
elif contains_any_substring(strip_ol(lines[5]), ["skinny"]):
    result = 'Skinny figure.'
elif contains_any_substring(strip_ol(lines[5]), ["large", "overweight", "fat", "fuller figure"]):
    result = 'Large figure.'
else:
    result = '-'

lineout[5] = result
print( lineout[5] )

Normal figure.


In [148]:
x1 = true_or_false(strip_ol(lines[6]))
x2 = true_or_false(strip_ol(lines[7]), ["hands are visible","yes"])
x3 = true_or_false(strip_ol(lines[8]))

print(f"{x1} {x2} {x3}")

True True False


In [147]:
result = remove_matches(strip_ol(lines[9]), ["The woman's ","The woman is ","The woman "]).capitalize()
lineout[9] = result
print( lineout[9] )

Lying on her back


In [146]:
#print( lines[10] )
if not contains_any_substring(strip_ol(lines[10]), ['not visible']):
    result = remove_matches(strip_ol(lines[10]), ["The woman's ","The woman is ","The woman ", "likely engaging in "])
    result = string__return_left_from_match(result, [', possibly',', exposing it to'], '.').capitalize()
else:
    result = '-'


lineout[10] = result
print( lineout[10] )

Hands are resting on her thighs, adding to the sensual and alluring atmosphere of the image.
